#### Importing Libraries

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

#### Hyperparameters

In [ ]:
batch_size = 64
classes = 6
learning_rate = 0.001
epochs = 64
dropout_rate = 0.3

#### Initializing CUDA

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Preprocessing 

In [4]:
# Computed Values from values.py
# Mean: [0.6502255201339722, 0.626167893409729, 0.5942673683166504]
# Std: [0.26641741394996643, 0.263718843460083, 0.277700275182724]

In [5]:
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.6502, 0.6261, 0.5942],
                                                          std=[0.2664, 0.2637, 0.2777])
                                         ])

train_transforms = transforms.Compose([transforms.Resize((224, 224)), 
                                        transforms.RandomRotation(20), 
                                        transforms.RandomResizedCrop(224, scale=(0.8, 1.2)), 
                                        transforms.RandomHorizontalFlip(), 
                                        transforms.RandomAffine(degrees=0, shear=20), 
                                        transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.5), 
                                        transforms.ToTensor(), 
                                        transforms.Normalize(mean=[0.6502, 0.6261, 0.5942],
                                                          std=[0.2664, 0.2637, 0.2777]) 
                                        ])

#### Importing Dataset

In [6]:
train_dataset = ImageFolder(root="./dataset/training", transform=train_transforms)
test_dataset = ImageFolder(root="./dataset/testing", transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

print(f"Training dataset size: {len(train_dataset)} images")
print(f"Testing dataset size: {len(test_dataset)} images")

Training dataset size: 5568 images
Testing dataset size: 1835 images


#### Defining neural network

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=5, img_height=224, img_width=224, dropout_rate=0.4):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.ddropropout = nn.Dropout(p=out_rate)  # Dropout layer

        final_size = img_height // 16  
        final_size = img_width // 16
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * final_size * final_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc2(x)  
        return x

#### Setting up the model

In [10]:
# Init Model
model = CNN(classes)
# Init Loss Function
criterion = nn.CrossEntropyLoss()
# Init Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
# Init Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,verbose=True)
total_step = len(train_loader)

c:\Users\Tushar\miniconda3\envs\torch-cuda\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


#### Fitting the Model

In [11]:
torch.cuda.empty_cache()
model.to(device)

for epoch in range(epochs):
    epoch_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader) 
    scheduler.step(avg_loss)  

    print('Epoch [{}/{}], Loss: {:.4f}, Learning Rate: {:.6f}'.format(epoch+1, epochs, loss.item(),optimizer.param_groups[0]["lr"]))

Epoch [1/28], Loss: 1.1935, Learning Rate: 0.001000
Epoch [2/28], Loss: 1.3656, Learning Rate: 0.001000
Epoch [3/28], Loss: 1.6966, Learning Rate: 0.001000
Epoch [4/28], Loss: 0.6973, Learning Rate: 0.001000
Epoch [5/28], Loss: 0.7978, Learning Rate: 0.001000
Epoch [6/28], Loss: 0.7625, Learning Rate: 0.001000
Epoch [7/28], Loss: 1.0901, Learning Rate: 0.001000
Epoch [8/28], Loss: 0.3958, Learning Rate: 0.001000
Epoch [9/28], Loss: 0.5629, Learning Rate: 0.001000
Epoch [10/28], Loss: 1.5292, Learning Rate: 0.001000
Epoch [11/28], Loss: 0.5533, Learning Rate: 0.001000
Epoch [12/28], Loss: 2.0296, Learning Rate: 0.001000
Epoch [13/28], Loss: 0.4789, Learning Rate: 0.001000
Epoch [14/28], Loss: 0.9219, Learning Rate: 0.001000
Epoch [15/28], Loss: 1.1482, Learning Rate: 0.001000
Epoch [16/28], Loss: 0.3371, Learning Rate: 0.001000
Epoch [17/28], Loss: 0.7376, Learning Rate: 0.001000
Epoch [18/28], Loss: 0.4491, Learning Rate: 0.001000
Epoch [19/28], Loss: 0.5375, Learning Rate: 0.001000
Ep

#### Testing

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {:.3f}%'.format(1835, 100 * correct / total))

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {:.3f}%'.format(5568, 100 * correct / total))

Accuracy of the network on the 1835 test images: 77.057%
Accuracy of the network on the 5568 train images: 85.129%


In [14]:
torch.save(model,"ecovis alpha-v0.9.pkl")

In [16]:
torch.save(model.state_dict(), "ecovis.pth")